In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.options.display.max_colwidth = 100
plt.style.use('ggplot')

#### Import the data
This version of the CSV has all categorical variables encoded, in addition to newly created 'feature engineered' columns.

In [4]:
df = pd.read_csv('Ames_House_Price_FeatureEngineered_Encoded.csv', index_col=0)

## Feature Sets
Each list is sorted in order of Gini Importance (Random-Forest)

In [6]:
# all features before feature reduction
features_all = ['overall_qual', 'gr_liv_area', '1st_flr_sf', 'total_bsmt_sf', 'garage_area', 'bsmt_fin_sf1', 'lot_area', 'house_age_at_sale', 'construction_age_at_sale', 'garage_cars', 'mas_vnr_area', 'bsmt_qual', '2nd_flr_sf', 'bsmt_fin_type1', 'kitchen_qual', 'lot_frontage', 'garage_finish', 'bsmt_unf_sf', 'overall_cond', 'open_porch_sf', 'tot_rms_abv_grd', 'wood_deck_sf', 'bsmt_exposure', 'mo_sold', 'fireplace_qu', 'screen_porch', 'fireplaces', 'full_bath', 'ms_zoning_rm', 'exter_qual', 'bsmt_full_bath', 'central_air', 'heating_qc', 'bedroom_abv_gr', 'garage_type_detchd', 'lot_shape', 'half_bath', 'ms_sub_class_60', 'neighborhood_crawfor', 'enclosed_porch', 'roof_style_hip', 'land_slope', 'bsmt_cond', 'exterior1st_metalsd', 'sale_condition_family', 'functional', 'foundation_pconc', 'yr_sold_2007', 'paved_drive', 'lot_config_corner', 'bsmt_fin_sf2', 'yr_sold_2008', 'yr_sold_2009', 'garage_cond', 'fence', 'exter_cond', 'land_contour_hls', 'exterior1st_wd sdng', 'mas_vnr_type_stone', 'bsmt_fin_type2', 'neighborhood_noridge', 'neighborhood_collgcr', 'kitchen_abv_gr', 'near_main_rd', 'exterior1st_hdboard', 'lot_config_culdsac', 'mas_vnr_type_brkface', 'neighborhood_oldtown', 'roof_style_mansard', 'yr_sold_2010', 'exterior1st_cemntbd', 'sale_condition_partial', 'sale_type_new', 'ms_sub_class_30', 'exterior1st_brkface', 'garage_qual', 'neighborhood_edwards', 'house_style_1.5fin', 'neighborhood_stonebr', 'neighborhood_veenker', 'foundation_brktil', 'neighborhood_somerst', 'bsmt_half_bath', 'sale_condition_abnorml', 'neighborhood_nridght', 'house_style_2story', 'neighborhood_nwames', 'ms_zoning_fv', 'near_pos', 'ms_sub_class_70', 'land_contour_low', 'neighborhood_clearcr', 'alley_pave', 'ms_sub_class_75', 'exterior1st_plywood', 'electrical', 'neighborhood_sawyerw', '3_ssn_porch', 'ms_sub_class_50', 'neighborhood_timber', 'house_style_2.5unf', 'neighborhood_gilbert', 'low_qual_fin_sf', 'roof_style_flat', 'neighborhood_grnhill', 'house_style_slvl', 'ms_sub_class_120', 'land_contour_bnk', 'neighborhood_mitchel', 'near_rr', 'ms_sub_class_90', 'bldg_type_twnhse', 'neighborhood_sawyer', 'sale_type_oth', 'roof_style_gambrel', 'lot_config_fr3', 'garage_type_builtin', 'alley_grvl', 'neighborhood_brkside', 'lot_config_fr2', 'ms_sub_class_80', 'misc_feature_shed', 'foundation_slab', 'misc_val', 'ms_sub_class_190', 'sale_type_cod', 'ms_zoning_rh', 'garage_type_na', 'exterior1st_precast', 'neighborhood_swisu', 'neighborhood_idotrr', 'bldg_type_duplex', 'exterior1st_stucco', 'bldg_type_twnhs', 'neighborhood_npkvill', 'bldg_type_2fmcon', 'exterior1st_wdshing', 'house_style_2.5fin', 'ms_sub_class_160', 'exterior1st_asbshng', 'pool_area', 'ms_zoning_c (all)', 'house_style_sfoyer', 'neighborhood_meadowv', 'ms_sub_class_85', 'sale_type_conld', 'garage_type_2types', 'sale_type_conlw', 'mas_vnr_type_brkcmn', 'exterior1st_brkcomm', 'garage_type_carport', 'sale_type_con', 'garage_type_basment', 'ms_sub_class_40', 'sale_type_cwd', 'roof_style_shed', 'misc_feature_gar2', 'ms_sub_class_45', 'exterior1st_asphshn', 'neighborhood_blmngtn', 'neighborhood_brdale', 'sale_type_conli', 'foundation_stone', 'house_style_1.5unf', 'neighborhood_blueste', 'sale_condition_alloca', 'neighborhood_greens', 'foundation_wood', 'ms_zoning_a (agr)', 'neighborhood_landmrk', 'misc_feature_othr', 'exterior1st_imstucc', 'exterior1st_cblock', 'ms_zoning_i (all)', 'ms_sub_class_180', 'misc_feature_tenc', 'sale_type_vwd', 'sale_condition_adjland', 'ms_sub_class_150']

In [7]:
# removed features with variance threshold below 0.01
reduced_var = ['overall_qual', 'gr_liv_area', '1st_flr_sf', 'total_bsmt_sf', 'garage_area', 'bsmt_fin_sf1', 'house_age_at_sale', 'construction_age_at_sale', 'mas_vnr_area', 'garage_cars', 'bsmt_qual', '2nd_flr_sf', 'bsmt_unf_sf', 'kitchen_qual', 'bsmt_fin_type1', 'garage_finish', 'overall_cond', 'tot_rms_abv_grd', 'mo_sold', 'bsmt_exposure', 'fireplaces', 'fireplace_qu', 'full_bath', 'ms_zoning_rm', 'exter_qual', 'heating_qc', 'bsmt_full_bath', 'bedroom_abv_gr', 'half_bath', 'lot_shape', 'central_air', 'neighborhood_crawfor', 'garage_type_detchd', 'ms_sub_class_60', 'bsmt_cond', 'land_slope', 'yr_sold_2007', 'functional', 'roof_style_hip', 'fence', 'bsmt_fin_sf2', 'foundation_pconc', 'exterior1st_metalsd', 'land_contour_hls', 'yr_sold_2008', 'bsmt_fin_type2', 'lot_config_culdsac', 'paved_drive', 'yr_sold_2009', 'exterior1st_cemntbd', 'lot_config_corner', 'garage_cond', 'mas_vnr_type_stone', 'neighborhood_collgcr', 'exterior1st_hdboard', 'neighborhood_oldtown', 'bsmt_half_bath', 'near_main_rd', 'exterior1st_brkface', 'mas_vnr_type_brkface', 'neighborhood_edwards', 'yr_sold_2010', 'exterior1st_wd sdng', 'sale_type_new', 'near_pos', 'garage_qual', 'ms_sub_class_30', 'neighborhood_stonebr', 'alley_pave', 'land_contour_low', 'sale_condition_partial', 'sale_condition_abnorml', 'foundation_brktil', 'house_style_2story', 'neighborhood_clearcr', 'neighborhood_somerst', 'house_style_1.5fin', 'neighborhood_noridge', 'ms_sub_class_50', 'neighborhood_nridght', 'neighborhood_nwames', 'exterior1st_plywood', 'electrical', 'ms_zoning_fv', 'neighborhood_sawyerw', 'neighborhood_timber', 'neighborhood_sawyer', 'neighborhood_gilbert', 'bldg_type_twnhse', 'ms_sub_class_70', 'ms_sub_class_90', 'near_rr', 'ms_sub_class_120', 'garage_type_builtin', 'neighborhood_brkside', 'land_contour_bnk', 'bldg_type_duplex', 'lot_config_fr2', 'ms_sub_class_160', 'alley_grvl', 'neighborhood_mitchel', 'ms_sub_class_80', 'house_style_slvl', 'misc_feature_shed', 'neighborhood_idotrr', 'exterior1st_wdshing', 'ms_sub_class_190', 'sale_type_cod', 'garage_type_na', 'exterior1st_stucco', 'neighborhood_swisu', 'foundation_slab', 'bldg_type_twnhs', 'house_style_sfoyer', 'bldg_type_2fmcon', 'exterior1st_asbshng', 'ms_sub_class_85', 'neighborhood_meadowv', 'neighborhood_brdale', 'garage_type_basment']

In [8]:
# removed/engineering features to reduce colinearity below 0.75 (r-value)
corr_75 = ['overall_qual', 'gr_liv_area', 'bsmt_fin_sf', 'garage_area', 'house_age_at_sale', 'mas_vnr_area', 'bsmt_qual', 'construction_age_at_sale', 'bsmt_unf_sf', 'fireplaces', 'mo_sold', 'garage_finish', 'kitchen_qual', 'exter_qual', 'overall_cond', 'bsmt_fin_type1', 'bsmt_exposure', 'full_bath', 'ms_zoning_rm', 'heating_qc', 'central_air', 'bedroom_abv_gr', 'half_bath', 'lot_shape', 'has_2nd_floor', 'garage_type_detchd', 'neighborhood_crawfor', 'garage_rating', 'bsmt_full_bath', 'roof_style_hip', 'exterior1st_hdboard', 'functional', 'foundation_pconc', 'lot_config_culdsac', 'sale_type_new', 'yr_sold_2007', 'neighborhood_stonebr', 'bsmt_cond', 'land_slope', 'fence', 'exterior1st_brkface', 'exterior1st_wd sdng', 'neighborhood_noridge', 'bsmt_fin_type2', 'land_contour_hls', 'mas_vnr_type_stone', 'paved_drive', 'exterior1st_cemntbd', 'yr_sold_2010', 'yr_sold_2009', 'yr_sold_2008', 'mas_vnr_type_brkface', 'exterior1st_metalsd', 'lot_config_corner', 'neighborhood_somerst', 'alley_pave', 'neighborhood_edwards', 'neighborhood_clearcr', 'neighborhood_oldtown', 'house_style_1.5fin', 'near_pos', 'land_contour_low', 'near_main_rd', 'bsmt_half_bath', 'foundation_brktil', 'neighborhood_collgcr', 'neighborhood_nwames', 'neighborhood_timber', 'neighborhood_nridght', 'sale_condition_abnorml', 'garage_type_builtin', 'bldg_type_twnhse', 'neighborhood_sawyerw', 'electrical', 'bldg_type_twnhs', 'exterior1st_plywood', 'bldg_type_duplex', 'lot_config_fr2', 'neighborhood_sawyer', 'house_style_slvl', 'near_rr', 'land_contour_bnk', 'neighborhood_brkside', 'neighborhood_mitchel', 'neighborhood_meadowv', 'neighborhood_swisu', 'neighborhood_idotrr', 'neighborhood_gilbert', 'bldg_type_2fmcon', 'alley_grvl', 'misc_feature_shed', 'sale_type_cod', 'foundation_slab', 'house_style_sfoyer', 'exterior1st_stucco', 'exterior1st_wdshing', 'neighborhood_brdale', 'exterior1st_asbshng', 'garage_type_basment']

In [9]:
# using VIF (variance inflation factor), reduced/engineered features so that VIF threshold is below 80
vif_80 = ['overall_qual', 'gr_liv_area', 'bsmt_fin_sf', 'garage_area', 'house_age_at_sale', 'mas_vnr_area', 'bsmt_qual', 'construction_age_at_sale', 'bsmt_unf_sf', 'fireplaces', 'kitchen_qual', 'mo_sold', 'garage_finish', 'overall_cond', 'bsmt_fin_type1', 'bsmt_exposure', 'ms_zoning_rm', 'full_bath', 'heating_qc', 'central_air', 'bedroom_abv_gr', 'half_bath', 'has_2nd_floor', 'garage_type_detchd', 'neighborhood_crawfor', 'bsmt_full_bath', 'garage_rating', 'roof_style_hip', 'foundation_pconc', 'sale_type_new', 'exterior1st_hdboard', 'lot_config_culdsac', 'neighborhood_noridge', 'yr_sold_2007', 'lot_config_corner', 'bsmt_fin_type2', 'exterior1st_brkface', 'fence', 'land_contour_hls', 'exterior1st_metalsd', 'neighborhood_stonebr', 'yr_sold_2010', 'near_pos', 'exterior1st_wd sdng', 'yr_sold_2008', 'mas_vnr_type_stone', 'neighborhood_somerst', 'yr_sold_2009', 'neighborhood_clearcr', 'neighborhood_edwards', 'mas_vnr_type_brkface', 'house_style_1.5fin', 'exterior1st_cemntbd', 'bsmt_half_bath', 'foundation_brktil', 'garage_type_builtin', 'alley_pave', 'neighborhood_oldtown', 'neighborhood_collgcr', 'neighborhood_nwames', 'near_main_rd', 'neighborhood_nridght', 'land_contour_low', 'sale_condition_abnorml', 'bldg_type_twnhse', 'bldg_type_twnhs', 'neighborhood_timber', 'neighborhood_sawyerw', 'land_contour_bnk', 'exterior1st_plywood', 'bldg_type_duplex', 'house_style_slvl', 'lot_config_fr2', 'near_rr', 'neighborhood_sawyer', 'bldg_type_2fmcon', 'neighborhood_brkside', 'neighborhood_mitchel', 'neighborhood_meadowv', 'neighborhood_gilbert', 'neighborhood_idotrr', 'alley_grvl', 'neighborhood_swisu', 'misc_feature_shed', 'sale_type_cod', 'foundation_slab', 'exterior1st_stucco', 'house_style_sfoyer', 'exterior1st_wdshing', 'neighborhood_brdale', 'exterior1st_asbshng', 'garage_type_basment']

In [10]:
# using VIF (variance inflation factor), reduced/engineered features so that VIF threshold is below 50
vif_50 = ['total_fin_sf', 'house_age_at_sale', 'garage_area', 'bsmt_unf_sf', 'construction_age_at_sale', 'mas_vnr_area', 'fireplaces', 'overall_cond', 'full_bath', 'mo_sold', 'bsmt_exposure', 'central_air', 'bsmt_fin_type1', 'bedroom_abv_gr', 'garage_finish', 'neighborhood_crawfor', 'heating_qc', 'roof_style_hip', 'garage_rating', 'garage_type_detchd', 'neighborhood_stonebr', 'half_bath', 'land_contour_hls', 'neighborhood_noridge', 'exterior1st_metalsd', 'bsmt_full_bath', 'exterior1st_brkface', 'ms_zoning_rm', 'yr_sold_2007', 'lot_config_culdsac', 'neighborhood_somerst', 'exterior1st_cemntbd', 'bsmt_fin_type2', 'fence', 'yr_sold_2008', 'lot_config_corner', 'neighborhood_nridght', 'neighborhood_sawyerw', 'near_pos', 'yr_sold_2009', 'mas_vnr_type_brkface', 'has_2nd_floor', 'near_main_rd', 'exterior1st_hdboard', 'bldg_type_twnhse', 'land_contour_low', 'neighborhood_edwards', 'sale_condition_abnorml', 'yr_sold_2010', 'neighborhood_clearcr', 'neighborhood_sawyer', 'neighborhood_nwames', 'neighborhood_collgcr', 'sale_type_new', 'exterior1st_wd sdng', 'bsmt_half_bath', 'bldg_type_duplex', 'mas_vnr_type_stone', 'land_contour_bnk', 'foundation_pconc', 'house_style_1.5fin', 'exterior1st_plywood', 'foundation_brktil', 'neighborhood_meadowv', 'neighborhood_timber', 'neighborhood_oldtown', 'garage_type_builtin', 'alley_pave', 'house_style_slvl', 'lot_config_fr2', 'near_rr', 'neighborhood_mitchel', 'exterior1st_wdshing', 'neighborhood_brkside', 'bldg_type_twnhs', 'neighborhood_idotrr', 'neighborhood_gilbert', 'sale_type_cod', 'bldg_type_2fmcon', 'house_style_sfoyer', 'alley_grvl', 'exterior1st_stucco', 'misc_feature_shed', 'neighborhood_swisu', 'neighborhood_brdale', 'exterior1st_asbshng', 'foundation_slab', 'garage_type_basment']